In [2]:
import pandas as pd
import numpy as np
import altair as alt

import json

from IPython.display import display, HTML

# Dataset

This `csv` file was downloaded from the **GoodReads** website as their public **API** is no longer supported.

In [3]:
goodreads_df = pd.read_csv('goodreads_export.csv')

print(f"Dataset Columns")
for col in goodreads_df.columns:
    print(f"- {col}")

Dataset Columns
- Book Id
- Title
- Author
- Author l-f
- Additional Authors
- ISBN
- ISBN13
- My Rating
- Average Rating
- Publisher
- Binding
- Number of Pages
- Year Published
- Original Publication Year
- Date Read
- Date Added
- Bookshelves
- Bookshelves with positions
- Exclusive Shelf
- My Review
- Spoiler
- Private Notes
- Read Count
- Owned Copies


## Data Cleaning

Consolidate `Authors`, `Publication Year`, `Bookshelves` in a single field.

In [4]:
goodreads_df['Original Publication Year'] = goodreads_df['Original Publication Year'].fillna(
    goodreads_df['Year Published']
).astype(int)

goodreads_df['Authors'] = (
    goodreads_df['Author'].str.split(', ')
    + goodreads_df['Additional Authors'].fillna('').str.split(', ')
).apply(lambda x: " | ".join(x))

goodreads_df['My Review'] = goodreads_df['My Review'].fillna(' ')

In [5]:
goodreads_df.drop(
    columns = [
        'Year Published',
        'Author', 'Author l-f', 'Additional Authors',
        'Binding', 'Bookshelves'
    ], inplace = True
)

goodreads_df.rename(
    columns = {
        'Original Publication Year': 'Year Published',
        'Bookshelves with positions': 'Bookshelves',
    }, inplace = True
)

goodreads_df['My Review'] = goodreads_df['My Review'].fillna('')

In [6]:
ordered_columns = [
    'Book Id', 'Title', 'ISBN', 'ISBN13',
    'Authors', 'Publisher', 'Year Published',
    'Date Added', 'Date Read', 'Number of Pages',
    'My Rating', 'Average Rating', 'My Review',
    'Spoiler', 'Private Notes',
    'Owned Copies', 'Exclusive Shelf', 'Read Count',
    'Bookshelves',
]

goodreads_df = goodreads_df[ordered_columns]
display(goodreads_df)

,Book Id,Title,ISBN,ISBN13,Authors,Publisher,Year Published,Date Added,Date Read,Number of Pages,My Rating,Average Rating,My Review,Spoiler,Private Notes,Owned Copies,Exclusive Shelf,Read Count,Bookshelves
0,122765395,Elon Musk,"=""1982181281""","=""9781982181284""",Walter Isaacson |,Simon & Schuster,2023,2025/06/22,NaN,688,0,4.34,,NaN,NaN,0,currently-reading,1,currently-reading (#2)
1,215180148,The BESS Book: A Cell-to-Grid Guide to Utility...,"=""""","=""9798218455095""",Drew Lebowitz | Sean Daly | Swetha Sundaram,"BESS Book, The",2024,2025/06/22,NaN,603,0,4.27,,NaN,NaN,0,currently-reading,1,currently-reading (#1)
2,62634115,"8 Rules of Love: How to Find It, Keep It, and ...","=""198218308X""","=""9781982183080""",Jay Shetty |,Simon & Schuster,2023,2025/06/22,NaN,352,0,3.91,,NaN,NaN,0,to-read,0,to-read (#6)
3,49195924,No Rules Rules: Netflix and the Culture of Rei...,"=""1984877879""","=""9781984877871""",Reed Hastings | Erin Meyer,Penguin Press,2020,2025/06/22,NaN,320,0,4.27,,NaN,NaN,0,to-read,0,to-read (#5)
4,26073005,The Grid: The Fraying Wires Between Americans ...,"=""1608196100""","=""9781608196104""",Gretchen Bakke |,Bloomsbury USA,2016,2023/05/15,2024/06/22,384,4,3.87,This book emphasizes the importance of 20th ce...,NaN,NaN,0,read,1,NaN
5,53138083,"Working Backwards: Insights, Stories, and Secr...","=""1250267595""","=""9781250267597""",Colin Bryar | Bill Carr,St. Martin's Press,2021,2022/07/16,2022/07/01,304,5,4.21,Really insightful look within Amazon's busines...,NaN,NaN,0,read,1,NaN
6,472331,Watchmen,"=""0930289234""","=""9780930289232""",Alan Moore | Dave Gibbons | John ...,DC Comics,1987,2022/07/16,2018/04/01,416,5,4.39,"This is somehow a superhero graphic novel, a c...",NaN,NaN,0,read,1,NaN
7,19145267,"Thinking, Fast and Slow","=""0385676522""","=""9780385676526""",Daniel Kahneman |,Doubleday Canada,2011,2022/08/07,NaN,512,0,4.17,,NaN,NaN,0,to-read,1,to-read (#4)
8,36147818,"Elon Musk: Tesla, SpaceX, and the Quest for a ...","=""""","=""""",Ashlee Vance |,Ecco,2015,2023/04/03,2023/02/01,392,4,4.13,,NaN,NaN,0,read,1,NaN
9,18492257,Lean Customer Development: Building Products Y...,"=""1449356354""","=""9781449356354""",Cindy Alvarez |,O'Reilly Media,2014,2022/07/16,2022/05/01,236,4,4.19,This book contains a really solid guideline fo...,NaN,NaN,0,read,1,NaN


In [7]:
goodreads_df.to_csv('goodreads.csv', index = False)
goodreads_df = pd.read_csv('goodreads.csv')

## Read Books

In [8]:
read_df = goodreads_df[(goodreads_df['Exclusive Shelf'] == 'read')]
current_df = goodreads_df[(goodreads_df['Exclusive Shelf'] == 'currently-reading')]
queue_df = goodreads_df[(goodreads_df['Exclusive Shelf'] == 'to-read')]

### Reviews

In [9]:
def generateRatingsBarchart(df) -> alt.Chart:
    formatted_df = df.copy()
    formatted_df["SummarizedTitle"] = formatted_df["Title"].str.split(":").str[0]
    formatted_df["Date Read"] = formatted_df["Date Read"].str.split('/').str[0].astype(int)

    formatted_df.rename(
        columns = {
            "My Rating": "Muntakim",
            "Average Rating": "GoodReads Average",
            "Date Read": "Year",
        },
        inplace = True
    )

    melted_df = pd.melt(
        formatted_df,
        id_vars = ["SummarizedTitle", "Title", "Year"],
        value_vars = ["Muntakim", "GoodReads Average"],
        var_name = "Reviewer",
        value_name = "Rating"
    )

    bar_chart = alt.Chart(melted_df).mark_bar(size = 10).encode(
        x = alt.X(
            "Rating:Q", title = "Rating",
            axis = alt.Axis(titleFontSize = 14, labelFontSize = 12),
            scale = alt.Scale(domain = [0, 5])
        ),
        y = alt.Y(
            "SummarizedTitle:N", title = "Book",
            sort = alt.EncodingSortField(
                field = "Rating", order = "descending"
            ),
            axis = alt.Axis(titleFontSize = 14, labelFontSize = 12, labelLimit = 0),
        ),
        color = alt.Color(
            "Reviewer:N",
            scale = alt.Scale(
                domain = ["Muntakim", "GoodReads Average"],
                range = ["seagreen", "orange"]
            ),
            legend = alt.Legend(
                title = "Reviewer",
                titleFontSize = 18, labelFontSize = 14,
                symbolType = "square", symbolSize = 100
            )
        ),
        yOffset = "Reviewer:N",
        tooltip = [
            alt.Tooltip("Title", title = "Title"),
            alt.Tooltip("Year", title = "Year Read"),
            alt.Tooltip("Reviewer", title = "Reviewer"),
            alt.Tooltip("Rating", title = "Rating"),

        ]
    ).properties(
        width = 300,
        height = 25 * len(melted_df["Title"].unique()),
        title = alt.Title(
            "Muntakim's Ratings",
            fontSize = 24,
        )
    )

    bar_chart.display()
    return bar_chart

ratings_chart = generateRatingsBarchart(read_df)

alt.Chart(...)

In [10]:
def generateYearBarchart(df) -> alt.Chart:
    agg_df = df.copy()
    agg_df['Year Read'] = agg_df['Date Read'].str.split('/').str[0].astype(int)

    years_df = pd.DataFrame({'Year': range(agg_df['Year Read'].min(), agg_df['Year Read'].max() + 1)})
    agg_df = agg_df.groupby('Year Read')\
        .agg(Books = ('Title', 'count'), Pages = ('Number of Pages', 'sum'))\
        .reset_index()\
        .rename(columns = {'Year Read': 'Year'})\

    agg_df = pd.merge(years_df, agg_df, on = 'Year', how = 'left').fillna(0)

    agg_df['Books'] = agg_df['Books'].astype(int)
    agg_df['Pages'] = agg_df['Pages'].astype(int)
    agg_df.sort_values(by = 'Year', inplace = True)

    year_chart = alt.Chart(agg_df).mark_bar().encode(
        x = alt.X('Year:N', title = 'Year', axis = alt.Axis(titleFontSize = 14, labelFontSize = 12, labelAngle = 0)),
        y = alt.Y('Books:Q', title = 'Books (#)', axis = alt.Axis(titleFontSize = 14, labelFontSize = 12)),
        color = alt.Color(
            'Pages:Q', title = 'Pages (#)', scale = alt.Scale(scheme = 'greens'),
            legend = alt.Legend(
                title = "Pages (#)",
                titleFontSize = 18, labelFontSize = 14,
            )
        ),
        tooltip = [
            alt.Tooltip('Year:O', title = 'Year'),
            alt.Tooltip('Books:Q', title = 'Books (#)'),
            alt.Tooltip('Pages:Q', title = 'Pages (#)'),
        ]
    ).properties(
        width = 250,
        height = 25 * df['Title'].nunique(),
        title = alt.Title(
            "Annual Summary",
            fontSize = 24,
        )
    )

    year_chart.display()
    return year_chart

year_chart = generateYearBarchart(read_df)

alt.Chart(...)

In [11]:
def generateDashboard() -> alt.HConcatChart:
    dashboard = alt.hconcat(
        ratings_chart,
        year_chart,
        spacing = 20
    ).properties(
        title = alt.TitleParams(
            text = f"Muntakim's GoodReads Dashboard",
            anchor = 'middle', fontSize = 40
        )
    )

    dashboard.display()
    return dashboard

dashboard = generateDashboard()

alt.HConcatChart(...)

In [12]:
filename = "Muntakim_Dashboard"
with open(f"Charts/{filename}.json", 'w') as f:
    json.dump(dashboard.to_dict(), f, indent = 2)
dashboard.save(f"Charts/{filename}.png") # Save as PNG (Static)
dashboard.save(f"Charts/{filename}.svg") # Save as SVG (Vector)